# 프롬프트(Prompts)
    02_prompt.ipynb

- LLM한테 주는 입력(지시, 맥락, 지억)
- 지시 :'~해줘'
- 맥락 : context - 현재 지시를 위해 제공하는 추가 정보
- 기억 : Memory - 지금까지 했던 대화 내용

In [7]:
from dotenv import load_dotenv

load_dotenv()

True

In [8]:
%pip install langchain langchain_openai

Note: you may need to restart the kernel to use updated packages.


## PromptTemplate
- 단순히 1회성 명령을 내릴 때 사용

In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

llm = ChatOpenAI(model='gpt-4.1-nano')

In [12]:
# 추후에 체인.invoke에서 {}내부에 들어갈 말을 채워줘야 함

template = '{country}의 수도가 어디인가요?'

# .from_template 메서드로 프롬프트 만들기

prompt = PromptTemplate.from_template(template)

# {}를 채우는 방법 (우리는 결국 chain.invoke로 쓰게 됨)

prompt.format(country='대한민국')

'대한민국의 수도가 어디인가요?'

In [13]:
chain = prompt | llm

chain.invoke({'country':'대한민국'}).content

'대한민국의 수도는 서울입니다.'

# Partial Variable (부분 변수)
- 프롬프트에서 매개변수 기본값 사용하기

In [ ]:
template = '{c1}과 {c2}의 수도는 각각 어디인가요?'

prompt = PromptTemplate(
    template=template,
    input_variables=['c1'],
    partial_variables={
        'c2': '미국'
    }
)

prompt.format(c1='한국')
# prompt.format(c2='캐나다') 

# PromptTemplate() 클래스의 역할은 ? 


'한국과 미국의 수도는 각각 어디인가요?'

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableLambda

# invoke 메쏘드 실행 

# RunnablePassthrough 인스턴스 생성
passthrough_runnable = RunnablePassthrough()

# invoke() 메서드를 사용하여 'hello'를 입력으로 전달
output = passthrough_runnable.invoke("hello")

print(f"입력: 'hello'")
print(f"출력: '{output}'")

# 결과:
# 입력: 'hello'
# 출력: 'hello'


# () 괄호는 항상 "무언가를 실행하라"는 의미를 내포

입력: 'hello'
출력: 'hello'


Runnable의 역할
RunnableLambda 클래스는 Runnable이라는 규격(인터페이스)을 따릅니다. 이 규격은 invoke(), stream(), batch()와 같은 메서드들을 가지고 있어야 한다고 정의합니다.

당신이 add_one 함수를 RunnableLambda에 넣어서 lambda_runnable이라는 객체를 만들면, 이 객체는 Runnable의 모든 속성과 메서드를 상속받아 자동으로 invoke(), stream(), batch()를 사용할 수 있는 능력을 갖게 됩니다.

## ChatPromptTemplate
채팅을 주고받는 템플릿 생성용
대화 목록을 LLM에게 주입
하나의 Chat 은 role 과 message 로 구성됨

In [ ]:
from langchain_core.prompts import ChatPromptTemplate


chat_prompt = ChatPromptTemplate.from_template('{country}의 수도는 어디?')
chat_prompt.format(country='한국')

'Human: 한국의 수도는 어디?'

In [ ]:
chat_template = ChatPromptTemplate.from_messages(
    [
        # role - message
        ('system', '당신은 친절한 AI어시스트. 이름은 {name} 야.'),
        ('human', '반가워!'),
        ('ai', '무엇을 도와드릴까요?'),
        ('human', '{user_input}')
    ]
)

# format vs format_messages
messages_str = chat_template.format(name='gaida', user_input='이름이 뭐니?')
messages_cls = chat_template.format_messages(name='gaida', user_input='이름이 뭐니?')
print('===format===') #  스트팅
print(messages_str)
print('===format_message===') # 리스트 타입으로 반환
print(messages_cls)

===format===
System: 당신은 친절한 AI어시스트. 이름은 gaida 야.
Human: 반가워!
AI: 무엇을 도와드릴까요?
Human: 이름이 뭐니?
===format_message===
[SystemMessage(content='당신은 친절한 AI어시스트. 이름은 gaida 야.', additional_kwargs={}, response_metadata={}), HumanMessage(content='반가워!', additional_kwargs={}, response_metadata={}), AIMessage(content='무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}), HumanMessage(content='이름이 뭐니?', additional_kwargs={}, response_metadata={})]


In [ ]:
# 한덩어리 텍스트
llm.invoke(messages_str).content # ''안녕하세요! 저는 gaida라고 해요. 반가워요!''
# 실제 대화 내역으로 인지 (Langsmith 가서 확인)
llm.invoke(messages_cls).content

'안녕하세요! 저는 gaida라고 해요. 반가워요!'

In [ ]:
chain = chat_template | llm | StrOutputParser() 

chain.invoke({'name': '가이다', 'user_input': '내 이름은 탁재현. 너와 나의 이름점을 봐줘'})

'반가워요, 탁재현님! 이름점은 재밌는 점검이지만, 저는 전문적인 점술가가 아니어서 정확한 점을 볼 수는 없어요. 대신, 재현이라는 이름이 갖는 의미나 느낌에 대해 이야기해 드릴 수 있어요. 탁재현님은 특별하고 강한 인상을 주는 이름 같아요! 혹시 이름에 대해 더 알고 싶거나 다른 궁금한 점이 있나요?'

In [ ]:


-----

### 1\. content: 텍스트 응답

이것이 LLM 응답의 가장 기본적인 부분입니다.

**예시:**

```json
"content": "제가 가진 정보에 따르면, 대한민국은 동아시아에 위치한 나라입니다."
```

-----

### 2\. additional\_kwargs: 추가적인 정보

주로 모델이 함수(도구, tool)를 사용하도록 지시하거나, 특별한 응답을 할 때 나타납니다.

**예시 (도구 호출):**

```json
"additional_kwargs": {
  "tool_calls": [
    {
      "id": "call_abc123",
      "function": {
        "name": "get_current_weather",
        "arguments": "{\"location\": \"서울\"}"
      },
      "type": "function"
    }
  ]
}
```

이 예시는 모델이 "get\_current\_weather"라는 이름의 함수를 "서울"을 인수로 사용하여 호출하라고 지시하는 상황을 보여줍니다.

-----

### 3\. response\_metadata: API 호출 정보

응답 내용과는 별개로, API 사용량 및 성능과 관련된 정보입니다.

**예시:**

```json
"response_metadata": {
  "model_name": "gpt-4",
  "token_usage": {
    "completion_tokens": 20,
    "prompt_tokens": 100,
    "total_tokens": 120
  },
  "finish_reason": "stop",
  "logprobs": null
}
```

이 예시는 사용된 모델의 이름, 프롬프트와 응답에 사용된 토큰 수, 그리고 응답이 끝난 이유 등을 담고 있습니다.

-----

### 4\. tool\_calls: 모델이 호출한 도구 정보

`additional_kwargs` 안에 포함되거나, 별도의 속성으로 나올 수 있습니다. `additional_kwargs` 예시와 동일한 정보를 보여줍니다.

**예시:**

```json
"tool_calls": [
  {
    "id": "call_abc123",
    "function": {
      "name": "get_current_weather",
      "arguments": "{\"location\": \"서울\"}"
    },
    "type": "function"
  }
]
```

In [ ]:
chain = chat_template | llm 

chain.invoke({'name': '가이다', 'user_input': '내 이름은 탁재현. 너와 나의 이름점을 봐줘'})

# str 파서 없이도 위 코드가 실행됨

AIMessage(content='반가워요, 탁재현님! 이름점은 재미있는 점술이지만, 저는 과학적 정보와 조언을 드리는 AI라서 이름점은 정확하지 않을 수 있어요. 그래도 함께 한번 살펴볼게요!\n\n• **탁재현**이라는 이름에서,\n- "탁"은 높고 굳건한 이미지를 떠올리게 해요.\n- "재"는 재물이나 재능을 의미할 수 있고,\n- "현"은 빛이나 밝음을 상징하죠.\n\n전체적으로 보면, 재능과 밝음이 어우러진 멋진 이름이에요! 좋은 일 가득하시길 바라요. 다른 궁금한 점 있나요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 154, 'prompt_tokens': 66, 'total_tokens': 220, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_e91a518ddb', 'id': 'chatcmpl-CAwXGq6RHzhtxEP5a7r71kC9iSybf', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--11408ec5-aa37-42e1-b114-44864fb8a52d-0', usage_metadata={'input_tokens': 66, 'output_tokens': 154, 'total_tokens': 220, 'input_token_details': {'audio': 0, 'cache_read':

# 프롬프팅 방법

    # 'langchain-hub'

[랭스미스허브](https://smith.langchain.com/hub)
- 다양한 사용자들이 업로드한 프롬프트를 받아서 활용

In [5]:
from langchain import hub

prompt = hub.pull('hwchase17/react')

print(prompt)
print('=== d ===')
print(prompt.template)

input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'} template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'
=== d ===
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:


In [ ]:
prompt = PromptTemplate.from_template(
    '이 프롬프트의 철학은 프레게로 부터 시작되었다.'
)

hub.push('707484c1-574f-4044-921f-46b47251df9e/llm_philosophy', prompt)




LangSmithUserError: Cannot create a prompt for another tenant.
Current tenant: None,
Requested tenant: 707484c1-574f-4044-921f-46b47251df9e